# Example of use of functions.py script
17/08/2021   
Gabriel Besombes   
#     

## Import needed packages
---

`sys` is needed to add the path to the functions.py file to the path variable

In [1]:
import sys
import opensilexClientToolsPython
import json

Insert the new path at position 1. Position 0 is the script path (or '' in REPL)    
Change "C:\\Users\\besombes\\Documents\\gitlab\\opensilex-ws-python-client" to the corresponding path to the cloned repository on your computer.

In [2]:
sys.path.insert(1, "C:\\Users\\besombes\\Documents\\gitlab\\opensilex-ws-python-client")

The file can now be imported

In [3]:
import functions

___
#   

## Connect to opensilex
___

__WARNING__ : You need to have a JSON file with your credentials (for me it's credentials.json and contains the following :   
`{
    "identifier":"guest@opensilex.org",
    "password":"guest_password",
    "host":"server_ip:server_port/rest"
}`)   
or to pass your credentials directly to `pythonClient.connect_to_opensilex_ws()`

In [4]:
with open('credentials.json') as f:
    credentials_dict = json.load(f)


In [5]:
pythonClient = opensilexClientToolsPython.ApiClient()
pythonClient.connect_to_opensilex_ws(**credentials_dict)

___
#    

## Migrate Variables from csv
___

To migrate variables from a csv a schema is needed. Here the default schema is used :

In [6]:
variables_schema = {
    'trait':'trait.uri',
    'trait_name':'trait.label',
    'entity':{
        'name':'entity.label',
        'uri':'entity.uri',
        'description':'entity.comment'
    },
    'characteristic':{
        'name':'characteristic.label',
        'uri':'characteristic.uri',
        'description':'characteristic.comment'
    },
    'method':{
        'name':'method.label',
        'uri':'method.uri',
        'description':'method.comment'
    },
    'unit':{
        'name':'unit.label',
        'uri':'unit.uri',
        'description':'unit.comment',
    },
    'uri':'variable.uri',
    'name':'variable.label',
    'description':'variable.description',
    'datatype':'variable.datatype',
    'alternative_name':'variable.alternative_name',
    'time_interval':'variable.timeinterval',
    'sampling_interval':'variable.sampleinterval'
}

Trying to import example variables from `.\csv_example\extended_variables_testing.csv`   
__NOTE__ : This will most likely fail as these variables probably already exist

In [7]:
res_csv = functions.migrate_variables_from_csv(
    pythonClient=pythonClient, 
    csv_path=".\\csv_example\\extended_variables_testing.csv",
    variables_schema=variables_schema
)
res_csv

2021-08-23 14:52:58,254 INFO         [functions.py:558] Update mode variable is set to False


C:\Users\besombes\anaconda3\envs\PyR\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
2021-08-23 14:52:59,348 INFO         [functions.py:875] Object created: {'uri': 'test:set/test_entity_uri_G_1', 'name': 'test_entity_name_G_1', 'description': 'test_entity_comment_G_1', 'exact_match': [], 'close_match': [], 'broad_match': [], 'narrow_match': []}

C:\Users\besombes\anaconda3\envs\PyR\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must spec

,trait,trait_name,entity,characteristic,method,unit,uri,name,description,datatype,alternative_name,time_interval,sampling_interval,exact_match,close_match,broad_match,narrow_match
0,test:set/test_trait_uri_G_1,test_trait_name_G_1,"{'name': 'test_entity_name_G_1', 'uri': 'test:...","{'name': 'test_characteristic_name_G_1',\n 'ur...","{'name': 'test_method_name_G_1', 'uri': 'test:...","{'name': 'test_unit_name_G_1', 'uri': 'test:se...",test:set/test_variable_uri_G_1,test_variable_name_G_1,test_variable_comment_G_1,http://www.w3.org/2001/XMLSchema#decimal,test_variable_alt_name_G_1,Millisecond,Millimeter,[],[],[],[]
1,test:set/test_trait_uri_G_2,test_trait_name_G_2,"{'name': 'test_entity_name_G_2', 'uri': 'test:...","{'name': 'test_characteristic_name_G_2',\n 'ur...","{'name': 'test_method_name_G_2', 'uri': 'test:...","{'name': 'test_unit_name_G_2', 'uri': 'test:se...",test:set/test_variable_uri_G_2,test_variable_name_G_2,test_variable_comment_G_2,http://www.w3.org/2001/XMLSchema#integer,test_variable_alt_name_G_2,Second,Centimeter,[],[],[],[]
2,test:set/test_trait_uri_G_3,test_trait_name_G_3,"{'name': 'test_entity_name_G_3', 'uri': 'test:...","{'name': 'test_characteristic_name_G_3',\n 'ur...","{'name': 'test_method_name_G_3', 'uri': 'test:...","{'name': 'test_unit_name_G_3', 'uri': 'test:se...",test:set/test_variable_uri_G_3,test_variable_name_G_3,test_variable_comment_G_3,http://www.w3.org/2001/XMLSchema#date,test_variable_alt_name_G_3,Minutes,Meter,[],[],[],[]
3,test:set/test_trait_uri_G_4,test_trait_name_G_4,"{'name': 'test_entity_name_G_4',\n 'uri': 'tes...","{'name': 'test_characteristic_name_G_4',\n 'ur...","{'name': 'test_method_name_G_4', 'uri': 'test:...","{'name': 'test_unit_name_G_4', 'uri': 'test:se...",test:set/test_variable_uri_G_4,test_variable_name_G_4,test_variable_comment_G_4,http://www.w3.org/2001/XMLSchema#string,test_variable_alt_name_G_4,Hour,Kilometer,[],[],[],[]
4,test:set/test_trait_uri_G_5,test_trait_name_G_5,False,test:set/test_characteristic_uri_G_5,test:set/test_method_uri_G_5,test:set/test_unit_uri_G_5,test:set/test_variable_uri_G_5,test_variable_name_G_5,test_variable_comment_G_5,http://www.w3.org/2001/XMLSchema#decimal,test_variable_alt_name_G_5,Millisecond,Millimeter,NaN,NaN,NaN,NaN
5,test:set/test_trait_uri_G_6,test_trait_name_G_6,"{'name': 'test_entity_name_G_6', 'uri': 'test:...","{'name': 'test_characteristic_name_G_6',\n 'ur...","{'name': 'test_method_name_G_6', 'uri': 'test:...","{'name': 'test_unit_name_G_6', 'uri': 'test:se...",test:set/test_variable_uri_G_6,test_variable_name_G_6,test_variable_comment_G_6,http://www.w3.org/2001/XMLSchema#integer,test_variable_alt_name_G_6,Second,Centimeter,[],[],[],[]
6,test:set/test_trait_uri_G_7,test_trait_name_G_7,"{'name': 'test_entity_name_G_7', 'uri': 'test:...","{'name': 'test_characteristic_name_G_7',\n 'ur...","{'name': 'test_method_name_G_7', 'uri': 'test:...","{'name': 'test_unit_name_G_7', 'uri': 'test:se...",test:set/test_variable_uri_G_7,test_variable_name_G_7,test_variable_comment_G_7,http://www.w3.org/2001/XMLSchema#date,test_variable_alt_name_G_7,Minutes,Meter,[],[],[],[]
7,test:set/test_trait_uri_G_8,test_trait_name_G_8,test:set/test_entity_uri_G_8,False,test:set/test_method_uri_G_8,test:set/test_unit_uri_G_8,test:set/test_variable_uri_G_8,test_variable_name_G_8,test_variable_comment_G_8,http://www.w3.org/2001/XMLSchema#string,test_variable_alt_name_G_8,Hour,Kilometer,NaN,NaN,NaN,NaN
8,test:set/test_trait_uri_G_9,test_trait_name_G_9,"{'name': 'test_entity_name_G_9', 'uri': 'test:...","{'name': 'test_characteristic_name_G_9',\n 'ur...","{'name': 'test_method_name_G_9', 'uri': 'test:...","{'name': 'test_unit_name_G_9', 'uri': 'test:se...",test:set/test_variable_uri_G_9,test_variable_name_G_9,test_variable_comment_G_9,http://www.w3.org/2001/XMLSchema#decimal,test_variable_alt_name_G_9,Millisecond,Millimeter,[],[],[],[]
9,None,None,"{'name': 'test_entity_name_G_10', 'uri': 'test...","{'name': 'test_characteristic_name_G_10',\n 'u...","{'na

In [8]:
res_csv.columns

Index(['trait', 'trait_name', 'entity', 'characteristic', 'method', 'unit',
       'uri', 'name', 'description', 'datatype', 'alternative_name',
       'time_interval', 'sampling_interval', 'exact_match', 'close_match',
       'broad_match', 'narrow_match'],
      dtype='object')

If you get an error the most likely cause is the format of your csv. There are two ways to solve this :
 * If you know how to use pandas you should try to open your csv with pandas and pass the created DataFrame to functions.migrate_variables().
 * If you don't know how to use pandas try to save your csv with comas "," as separators

If you want more details on the outputs you can check the debug.log file.

___
#    

## Migrate Variables from googlesheet
---

In [9]:
variables_schema = {
    'trait_name':'trait.label',
    'entity':{
        'name':'entity.label'
    },
    'characteristic':{
        'name':'characteristic.label'
    },
    'method':{
        'name':'method.label'
    },
    'unit':{
        'name':'unit.label',
        'alternative_symbol':'symbol_column'
    },
    'name':'variable.name',
    'datatype':'variable.datatype',
    'description':'variable.description'
}

In [10]:
res_sheet = functions.migrate_variables_from_googlesheet(
    pythonClient=pythonClient,
    spreadsheet_url='https://docs.google.com/spreadsheets/d/1mqmXkSekWhdde7Ic4baiw9R3jSkS90CwTuQbZoaUIVk',
    gid_number='794620992',
    variables_schema=variables_schema
)
res_sheet

2021-08-23 14:56:06,615 INFO         [functions.py:346] https://docs.google.com/spreadsheets/d/1mqmXkSekWhdde7Ic4baiw9R3jSkS90CwTuQbZoaUIVk/gviz/tq?tqx=out:csv&gid=794620992
2021-08-23 14:56:08,163 INFO         [functions.py:558] Update mode variable is set to False


C:\Users\besombes\anaconda3\envs\PyR\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
2021-08-23 14:56:09,153 INFO         [functions.py:842] Object {'name': 'Entity test Google NEWSheet'} at row 0 wasn't created as an object with that name already exists.
That object was skipped and will appear in the "already_existed.csv" file.
The object used instead is {'uri': 'test:set/variables#variable.entity.entity-test-google-newsheet', 'name': 'Entity test Google NEWSheet', 'descri

,trait,trait_name,entity,characteristic,method,unit,uri,name,description,datatype,alternative_name,time_interval,sampling_interval,exact_match,close_match,broad_match,narrow_match
0,None,None,"{'name': 'Entity test Google NEWSheet',\n 'uri...",{'name': 'Characteristic test Google NEWSheet'...,"{'name': 'Method test Google NEWSheet 2',\n 'u...","{'name': 'Unit test Google NEWSheet 2',\n 'uri...",test:set/variables#variable.variable-test-goog...,Variable test Google NEWSheet 2,test insertion variable Google NEWsheet script...,http://www.w3.org/2001/XMLSchema#decimal,None,None,None,[],[],[],[]


---
#     

## Get all variables from opensilex
___

In [6]:
res = functions.get_variables(pythonClient=pythonClient)

In [9]:
res

,trait.uri,trait.label,entity.label,entity.uri,entity.comment,characteristic.label,characteristic.uri,characteristic.comment,method.label,method.uri,...,unit.label,unit.uri,unit.comment,variable.uri,variable.label,variable.description,variable.datatype,variable.alternative_name,variable.timeinterval,variable.sampleinterval
0,None,None,Plant,test:set/variables#variable.entity.plant,None,Height,test:set/variables#characteristic.height,None,Manual,test:set/variables#variable.method.manual,...,Centimeter,test:set/variables#variable.unit.centimeter,None,test:set/variables#variable.plant_height_manua...,Plant_Height_Manual_Centimeter,None,http://www.w3.org/2001/XMLSchema#decimal,Plant_Height,None,None
1,http://www.opensilex.org/demo/id/traits/t001,Leaf_Area_Index,Plant,test:set/variables#variable.entity.plant,None,Leaf-Area-Index,test:set/variables#characteristic.leaf-area-index,None,LAI_Computation,http://www.opensilex.org/demo/id/methods/m001,...,LAI,http://www.opensilex.org/demo/id/units/u001,None,http://www.opensilex.org/demo/id/variables/v001,Leaf-Area_LAI-Computation_LAI,No description,http://www.w3.org/2001/XMLSchema#decimal,None,None,None
2,None,None,Air,test:set/variables#variable.entity.air,None,Humidity,test:set/variables#characteristic.humidity,None,Measurement,test:set/variables#variable.method.measurement,...,Pourcentage,test:set/variables#variable.unit.pourcentage,None,test:set/variables#variable.air_humidity_measu...,Air_Humidity_Measurement_Pourcentage,None,http://www.w3.org/2001/XMLSchema#integer,Air_Humidity,None,None
3,None,None,Air,test:set/variables#variable.entity.air,None,Temperature,test:set/variables#characteristic.temperature,None,Measurement,test:set/variables#variable.method.measurement,...,Degree Celius,test:set/variables#variable.unit.degree-celius,None,test:set/variables#variable.air_temperature_me...,Air_Temperature_Measurement_Degree_Celius,None,http://www.w3.org/2001/XMLSchema#decimal,Air_Temperature,None,None
4,None,None,Air,test:set/variables#variable.entity.air,None,BC-test-car,test:set/variables#characteristic.bc-test,insertion characteristique onglet,BC-test-methode,test:set/variables#variable.method.bc-test-met,...,BC-unite,test:set/variables#variable.unit.bc-unite,hxfxtg,test:set/variables#variable.bc-var2/1,BC-var2,None,http://www.w3.org/2001/XMLSchema#boolean,Air_BC-test-car,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,test:set/test_trait_uri_ISA_1,test_trait_name_ISA_1,test_entity_name_ISA_1,test:set/test_entity_uri_ISA_1,test_entity_comment_ISA_1,test_characteristic_name_ISA_1,test:set/test_characteristic_uri_ISA_1,test_characteristic_comment_ISA_1,test_method_name_ISA_1,test:set/test_method_uri_ISA_1,...,test_unit_name_ISA_1,test:set/test_unit_uri_ISA_1,test_unit_comment_ISA_1,test:set/test_variable_uri_ISA_1,test_variable_name_ISA_1,test_variable_comment_ISA_1,http://www.w3.org/2001/XMLSchema#decimal,test_variable_alt_name_ISA_1,Millisecond,Millimeter
273,test:set/test_trait_uri_ISA_2,test_trait_name_ISA_2,test_entity_name_ISA_2,test:set/test_entity_uri_ISA_2,test_entity_comment_ISA_2,test_characteristic_name_ISA_2,test:set/test_characteristic_uri_ISA_2,test_characteristic_comment_ISA_2,test_method_name_ISA_2,test:set/test_method_uri_ISA_2,...,test_unit_name_ISA_2,test:set/test_unit_uri_ISA_2,test_unit_comment_ISA_2,test:set/test_variable_uri_ISA_2,test_variable_name_ISA_2,test_variable_comment_ISA_2,http://www.w3.org/2001/XMLSchema#integer,test_variable_alt_name_ISA_2,Second,Centimeter
274,test:set/test_trait_uri_ISA_3,test_trait_name_ISA_3,test_entity_name_ISA_3,test:set/test_entity_uri_ISA_3,test_entity_comment_ISA_3,test_characteristic_name_ISA_3,test:set/test_characteristic_uri_ISA_3,test_characteristic_comment_ISA_3,test_method_name_ISA_3,test:set/test_method_uri_ISA_3,...,test_unit_name_ISA_3,test:set/test_unit_uri_ISA_3,test_unit_comment_ISA_3,test:set/test_variable_uri_ISA_3,test_variable_name_ISA_3,test_variable_comment

By default the variables are saved in "variables_export.csv"

___
#    